In [ ]:
!pip install ipywidgets
from IPython.display import display
import ipywidgets as widgets


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

# Đọc dataset
df = pd.read_csv("diabetes2.csv")

X = df.drop("Outcome", axis=1)
y = df["Outcome"]

numeric_features = X.columns

preprocessor = ColumnTransformer(
    transformers=[
        ("num", Pipeline([
            ("imputer", SimpleImputer(strategy="median")),
            ("scaler", StandardScaler())
        ]), numeric_features)
    ]
)

# Tách train/test
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# Dictionary chứa các mô hình
models = {
    "Logistic Regression": LogisticRegression(max_iter=1000),
    "Naive Bayes": GaussianNB(),
    "Random Forest": RandomForestClassifier(n_estimators=200),
    "KNN": KNeighborsClassifier(n_neighbors=5)
}


In [ ]:
# Dropdown chọn mô hình
model_dropdown = widgets.Dropdown(
    options=list(models.keys()),
    description='Model:',
    value="Random Forest"
)

# Nút Train
train_button = widgets.Button(
    description="Huấn luyện mô hình",
    button_style="success"
)

# Output text
output = widgets.Output()

# Ô nhập giá trị từng feature
input_boxes = {}
for col in X.columns:
    input_boxes[col] = widgets.FloatText(
        description=col,
        value=0.0
    )

predict_button = widgets.Button(
    description="Dự đoán",
    button_style="info"
)

# ==============================
# Hàm train model
# ==============================
def train_model(b):
    with output:
        output.clear_output()
        selected = model_dropdown.value
        clf = Pipeline([("preprocess", preprocessor),
                        ("model", models[selected])])
        clf.fit(X_train, y_train)
        acc = accuracy_score(y_test, clf.predict(X_test))
        print(f"🎉 Accuracy của {selected}: {acc:.3f}")
        train_model.clf = clf  # lưu mô hình đã train

# ==============================
# Hàm dự đoán
# ==============================
def predict_model(b):
    with output:
        output.clear_output()

        if not hasattr(train_model, "clf"):
            print("⚠️ Bạn cần huấn luyện mô hình trước!")
            return

        vals = [input_boxes[col].value for col in X.columns]

        # Chuyển list sang DataFrame
        input_df = pd.DataFrame([vals], columns=X.columns)

        pred = train_model.clf.predict(input_df)[0]

        print("🔮 Kết quả dự đoán:",
              "🟥 Có bệnh" if pred==1 else "🟩 Không bệnh")

train_button.on_click(train_model)
predict_button.on_click(predict_model)

# ==============================
# Hiển thị giao diện
# ==============================
display(model_dropdown)
display(train_button)

for col in X.columns:
    display(input_boxes[col])

display(predict_button)
display(output)


Dropdown(description='Model:', index=2, options=('Logistic Regression', 'Naive Bayes', 'Random Forest', 'KNN')…

Button(button_style='success', description='Huấn luyện mô hình', style=ButtonStyle())

FloatText(value=0.0, description='Pregnancies')

FloatText(value=0.0, description='Glucose')

FloatText(value=0.0, description='BloodPressure')

...

FloatText(value=0.0, description='BMI')

FloatText(value=0.0, description='DiabetesPedigreeFunction')

FloatText(value=0.0, description='Age')

Button(button_style='info', description='Dự đoán', style=ButtonStyle())

Output()